# सेमॅंटिक कर्नलमध्ये एजंट स्क्रॅचपॅडसह चॅट इतिहास कमी करणे

हे नोटबुक सेमॅंटिक कर्नलच्या चॅट इतिहास कमी करण्याच्या वैशिष्ट्याचा वापर कसा करायचा याचे प्रदर्शन करते, तसेच संभाषणांमध्ये संदर्भ टिकवण्यासाठी एजंट स्क्रॅचपॅडचा उपयोग कसा करायचा हे दाखवते. हे दीर्घ संभाषण हाताळण्यासाठी कार्यक्षम AI एजंट तयार करण्यासाठी आवश्यक आहे, जे टोकन मर्यादा ओलांडत नाहीत.

## तुम्ही काय शिकाल:
1. **चॅट इतिहास कमी करणे**: टोकन वापर व्यवस्थापित करण्यासाठी संभाषणाचा इतिहास आपोआप कसा संक्षेप करायचा
2. **एजंट स्क्रॅचपॅड**: वापरकर्त्याच्या पसंती आणि पूर्ण केलेल्या कार्यांचा मागोवा ठेवण्यासाठी एक सतत मेमरी प्रणाली
3. **टोकन वापर ट्रॅकिंग**: इतिहास कमी करण्यासह आणि त्याशिवाय टोकन वापर कसा बदलतो हे निरीक्षण करा

## पूर्वतयारी:
- पर्यावरणीय व्हेरिएबल्ससह Azure OpenAI सेटअप
- मागील धड्यांमधून मूलभूत एजंट संकल्पनांची समज


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## एजंट स्क्रॅचपॅड समजून घेणे

### एजंट स्क्रॅचपॅड म्हणजे काय?

**एजंट स्क्रॅचपॅड** ही एक स्थिर मेमरी प्रणाली आहे जी एजंट्स वापरतात:
- **पूर्ण केलेल्या कार्यांचा मागोवा ठेवण्यासाठी**: वापरकर्त्यासाठी काय केले आहे ते नोंदवणे
- **वापरकर्ता प्राधान्ये साठवण्यासाठी**: आवडी, नापसंती आणि आवश्यकता लक्षात ठेवणे
- **संदर्भ टिकवण्यासाठी**: महत्त्वाची माहिती संभाषणांमध्ये सहज उपलब्ध ठेवणे
- **अतिरिक्त पुनरावृत्ती कमी करण्यासाठी**: एकाच प्रश्नांची वारंवार विचारणा टाळणे

### हे कसे कार्य करते:
1. **लिहिण्याच्या प्रक्रिया**: एजंट नवीन माहिती शिकल्यानंतर स्क्रॅचपॅड अपडेट करतो
2. **वाचण्याच्या प्रक्रिया**: निर्णय घेताना एजंट स्क्रॅचपॅडचा सल्ला घेतो
3. **स्थिरता**: माहिती टिकून राहते, जरी चॅट इतिहास कमी झाला तरी

याचा विचार एजंटच्या वैयक्तिक नोटबुकसारखा करा, जो संभाषणाच्या इतिहासाला पूरक असतो.


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## एजंट स्क्रॅचपॅड प्लगइन तयार करा

हा प्लगइन एजंटला सततच्या स्क्रॅचपॅड फाइलमध्ये वाचण्याची आणि लिहिण्याची परवानगी देतो.


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## चॅट इतिहास संक्षेपक प्रारंभ करा

ChatHistorySummarizationReducer आपोआप संभाषणाचा इतिहास संक्षेपित करतो जेव्हा तो ठराविक मर्यादेपेक्षा जास्त होतो.


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## सुट्टी नियोजन एजंट तयार करा

हा एजंट वापरकर्त्यांना सुट्ट्यांचे नियोजन करण्यात मदत करेल आणि स्क्रॅचपॅडद्वारे संदर्भ टिकवून ठेवेल.


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immediately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## सुट्टी नियोजन संभाषण चालवा

आता पूर्ण संभाषण चालवूया ज्यामध्ये दाखवले आहे:
1. प्रारंभिक नियोजन विनंती
2. प्राधान्य गोळा करणे
3. प्रवासाचा कार्यक्रम तयार करणे
4. स्थान बदल
5. चॅट इतिहास कमी करणे
6. स्क्रॅचपॅडचा वापर


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## परिणामांचे विश्लेषण

आपल्या संभाषणादरम्यान काय घडले ते विश्लेषित करूया:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## मुख्य मुद्दे

### 1. चॅट इतिहास कमी करणे
- **स्वयंचलित प्रक्रिया**: संदेशांची संख्या ठरावीक मर्यादेपेक्षा जास्त झाल्यावर कमी करण्याची प्रक्रिया सुरू होते
- **टोकन बचत**: संक्षेपणानंतर टोकन वापरात लक्षणीय घट होते
- **संदर्भ टिकवणे**: महत्त्वाची माहिती संक्षेपांमध्ये टिकवली जाते

### 2. एजंट स्क्रॅचपॅडचे फायदे
- **सततची स्मृती**: वापरकर्त्याच्या पसंती इतिहास कमी झाल्यानंतरही टिकून राहते
- **कार्य ट्रॅकिंग**: एजंट पूर्ण केलेल्या कामाचा नोंद ठेवतो
- **उत्तम अनुभव**: पसंती पुन्हा सांगण्याची गरज नाही

### 3. टोकन वापराचे नमुने
- **रेखीय वाढ**: प्रत्येक संदेशासोबत टोकनची संख्या वाढते
- **लक्षणीय घट**: कमी केल्यानंतर टोकनची संख्या मोठ्या प्रमाणात कमी होते
- **शाश्वत संवाद**: मर्यादेत दीर्घ संवाद साधण्यास सक्षम बनवते


## साफसफाई

या डेमोदरम्यान तयार केलेली स्क्रॅचपॅड फाइल साफ करा:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# सारांश

अभिनंदन! तुम्ही यशस्वीपणे प्रगत संदर्भ व्यवस्थापन क्षमतांसह AI एजंट लागू केला आहे:

## तुम्ही काय शिकले:
- **चॅट इतिहास कमी करणे**: संभाषणांचे स्वयंचलित सारांश तयार करून टोकन मर्यादा व्यवस्थापित करणे
- **एजंट स्क्रॅचपॅड**: वापरकर्त्याच्या प्राधान्ये आणि पूर्ण केलेल्या कार्यांसाठी स्थिर मेमरी लागू करणे
- **टोकन व्यवस्थापन**: दीर्घ संभाषणांमध्ये टोकन वापर ट्रॅक करणे आणि अनुकूल करणे
- **संदर्भ टिकवणे**: संभाषण कमी करताना महत्त्वाची माहिती कायम ठेवणे

## वास्तविक-जगातील उपयोग:
- **ग्राहक सेवा बॉट्स**: सत्रांदरम्यान ग्राहकांच्या प्राधान्ये लक्षात ठेवणे
- **वैयक्तिक सहाय्यक**: चालू प्रकल्प आणि वापरकर्त्याच्या सवयींचा मागोवा ठेवणे
- **शैक्षणिक शिक्षक**: विद्यार्थ्यांची प्रगती आणि शिकण्याच्या प्राधान्ये लक्षात ठेवणे
- **आरोग्य सहाय्यक**: टोकन मर्यादा लक्षात ठेवून रुग्णांचा इतिहास जतन करणे

## पुढील पावले:
- अधिक प्रगत स्क्रॅचपॅड योजना लागू करा
- बहु-वापरकर्ता परिस्थितीसाठी डेटाबेस स्टोरेज जोडा
- डोमेन-विशिष्ट गरजांसाठी सानुकूल कमी करण्याच्या रणनीती तयार करा
- सेमॅंटिक मेमरी शोधासाठी व्हेक्टर डेटाबेससह एकत्र करा
- पूर्ण संदर्भासह काही दिवसांनी संभाषण पुन्हा सुरू करू शकणारे एजंट तयार करा



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
